In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18127571752818075432, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 12136498577433204080
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:83:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 13152840376108236552
 physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1", name: "/device:GPU:2"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 10071424781587159268
 physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:87:00.0, compute capability: 6.1", name: "/device:GPU:3"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 13119028079962416490
 physical_d

In [3]:
import keras
import tensorflow as tf
from keras import backend as K

print("Keras Version:", keras.__version__)
print("Tensorflow Version:", tf.__version__)
print("image dim ordering:", K.image_dim_ordering())

Using TensorFlow backend.


Keras Version: 2.2.4
Tensorflow Version: 1.4.0
image dim ordering: tf


In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
# from sklearn.metrics import log_loss
# import sys
# import time

import os
# import pandas as pd

import numpy as np
# from glob import glob
# import cv2
# import skimage
# from skimage.transform import resize

# import keras
# from keras import layers
# from keras import models
# from keras import optimizers
# from keras.models import load_model
from sklearn.metrics import accuracy_score
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
# from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
# from keras.layers import Input, concatenate
# from keras import optimizers, metrics, models
# from keras.layers import Input, Flatten, Dense


In [9]:
output_classes = 2
batch_size = 32
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
# epochs = 50

nb_train_samples = 632300
nb_validation_samples = 184732
nb_test_samples = 15744

In [6]:
train_dir = 'data/train/'
validation_dir = 'data/validation'
test_dir = 'data/test'

In [7]:
for root,dirs,files in os.walk(train_dir):
    print (root, len(files))

print("*"*30)
for root,dirs,files in os.walk(validation_dir):
    print (root, len(files))
    
print("*"*30)
for root,dirs,files in os.walk(test_dir):
    print (root, len(files))

data/train/ 0
data/train/normal 375900
data/train/cancer 256400
******************************
data/validation 0
data/validation/normal 110140
data/validation/cancer 74592
******************************
data/test 0
data/test/normal 9376
data/test/cancer 6368


In [8]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 632300 images belonging to 2 classes.
Found 184732 images belonging to 2 classes.
Found 15744 images belonging to 2 classes.


In [10]:
def vgg19_model():
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    prediction = Dense(output_classes, activation=tf.nn.softmax)(x)

    model = Model(inputs=base_model.input,outputs=prediction)
    return model

In [11]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
         'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
         'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [12]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

#     random_seed = np.random.seed(1132)

#     train_datagen = ImageDataGenerator(
#         rescale=1. / 255,
#         featurewise_center=True,
#         featurewise_std_normalization=True)

#     test_datagen = ImageDataGenerator(rescale=1. / 255)

#     train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     validation_generator = train_datagen.flow_from_directory(
#         validation_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     test_generator = test_datagen.flow_from_directory(
#         test_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = False,
#         class_mode='categorical')

   
    adam_opt = Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = vgg19_model()
    
    model.compile(optimizer = adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      epochs = 15,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,
      verbose = 1,
      callbacks=get_callbacks(params))
    
    score = model.evaluate_generator(validation_generator, steps=50)

    print ('Validation Loss: ', score[0])
    print ('Validation Accuracy: ',score[1])
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
    predict_class = np.argmax(predicts, axis=1)
    errors = np.where(predict_class != truth)[0]
    print("No of errors = {}/{}".format(len(errors),test_generator.samples))
    th= 0.3
    acc = accuracy_score(truth, predict_class > th)
    print("Test Accuracy: {:.4f}".format(acc*100))
    print("*_*" * 50)
    
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [13]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_loss', patience=5, verbose=1)]
    return callbacks

In [14]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'amsgrad': True, 'beta_1': 0.3, 'beta_2': 0.995, 'lr': 1e-05}
Epoch 1/15
19759/19759 [==============================] - 7787s 394ms/step - loss: 0.3708 - acc: 0.9228 - val_loss: 0.3090 - val_acc: 0.9437
Epoch 2/15
19759/19759 [==============================] - 7841s 397ms/step - loss: 0.2805 - acc: 0.9553 - val_loss: 0.3527 - val_acc: 0.9148
Epoch 3/15
19759/19759 [==============================] - 7758s 393ms/step - loss: 0.2465 - acc: 0.9651 - val_loss: 0.2694 - val_acc: 0.9519
Epoch 4/15
19759/19759 [==============================] - 7767s 393ms/step - loss: 0.2211 - acc: 0.9718 - val_loss: 0.2548 - val_acc: 0.9553
Epoch 5/15
19759/19759 [==============================] - 7761s 393ms/step - loss: 0.2009 - acc: 0.9771 - val_loss: 0.2453 - val_acc: 0.9566
Epoch 6/15
19759/19759 [==============================] - 7755s 392ms/step - loss: 0.1837 - acc: 0.9812 - val_loss: 0.2228 - val_acc: 0.9648
Epoch 7/15
19759/19759 [==============================] - 7753s 392ms/

KeyboardInterrupt: 